In [3]:
import catheat
import numpy as np
import pandas as pd 
from functools import reduce 

%matplotlib inline
import glob
import os.path

In [4]:
path_to_plates="/Users/giana.cirolia/Desktop/Nucleofection_Pipelne_Code/plate_csv"
path_to_plates

'/Users/giana.cirolia/Desktop/Nucleofection_Pipelne_Code/plate_csv'

In [5]:
all_csv=glob.glob("{}/*.csv".format(path_to_plates))
all_csv

['/Users/giana.cirolia/Desktop/Nucleofection_Pipelne_Code/plate_csv/cell_number.csv',
 '/Users/giana.cirolia/Desktop/Nucleofection_Pipelne_Code/plate_csv/lonza_nucleofection_buffer.csv']

In [6]:
dict_of_dfs={}

for file in all_csv:
        file_name=os.path.splitext(os.path.basename(file))[0] # split based on extension
        df=pd.read_csv(file)
        dict_of_dfs[file_name]=df
dict_of_dfs

{'cell_number':   row_letter        1        2        3        4        5        6        7  \
 0          A  1000000  1000000  1000000  1000000  1000000  1000000  1000000   
 1          B  1000000  1000000  1000000  1000000  1000000  1000000  1000000   
 2          C  1000000  1000000  1000000  1000000  1000000  1000000  1000000   
 3          D  1000000  1000000  1000000  1000000  1000000  1000000  1000000   
 4          E  1000000  1000000  1000000  1000000  1000000  1000000  1000000   
 5          F  1000000  1000000  1000000  1000000  1000000  1000000  1000000   
 6          G  1000000  1000000  1000000  1000000  1000000  1000000  1000000   
 7          H  1000000  1000000  1000000  1000000  1000000  1000000  1000000   
 
          8        9       10       11       12  
 0  1000000  1000000  1000000  1000000  1000000  
 1  1000000  1000000  1000000  1000000  1000000  
 2  1000000  1000000  1000000  1000000  1000000  
 3  1000000  1000000  1000000  1000000  1000000  
 4  1000000  

In [7]:
def remove_cols(df):
    ### define a function that will drop unwanted columns from your df along the column axis
    cols_to_drop = ['row_letter', 'col_num'] 
    removed_cols = df.drop(cols_to_drop, axis=1)
    return removed_cols

tidy_files={} ## set and empty dictionary to which we will add key value pairs, your file name and the long format of a given 96 well plate
for file_name, data_frame in dict_of_dfs.items():
    df=pd.melt(data_frame, id_vars='row_letter', var_name='col_num', value_name=file_name) # turn 96 well format into long format, returning a df with a list of values for each well in your 96 well plate.  
    df["well_id_96"]=df['row_letter'] + df['col_num'] # make a new column to combine row letter with col number. 
    tidy_files[file_name]= df # pass the df into the remove cols function to remove col for row letter and col number and add to dictionary with asoociated file name. 

tidy_files

{'cell_number':    row_letter col_num  cell_number well_id_96
 0           A       1      1000000         A1
 1           B       1      1000000         B1
 2           C       1      1000000         C1
 3           D       1      1000000         D1
 4           E       1      1000000         E1
 5           F       1      1000000         F1
 6           G       1      1000000         G1
 7           H       1      1000000         H1
 8           A       2      1000000         A2
 9           B       2      1000000         B2
 10          C       2      1000000         C2
 11          D       2      1000000         D2
 12          E       2      1000000         E2
 13          F       2      1000000         F2
 14          G       2      1000000         G2
 15          H       2      1000000         H2
 16          A       3      1000000         A3
 17          B       3      1000000         B3
 18          C       3      1000000         C3
 19          D       3      1000000         D

In [8]:
def remove_cols(df):
    ### define a function that will drop unwanted columns from your df along the column axis
    cols_to_drop = ['row_letter', 'col_num'] 
    removed_cols = df.drop(cols_to_drop, axis=1)
    return removed_cols

tidy_files={} ## set and empty dictionary to which we will add key value pairs, your file name and the long format of a given 96 well plate
for file_name, data_frame in dict_of_dfs.items():
    df=pd.melt(data_frame, id_vars='row_letter', var_name='col_num', value_name=file_name) # turn 96 well format into long format, returning a df with a list of values for each well in your 96 well plate.  
    df["well_id_96"]=df['row_letter'] + df['col_num'] # make a new column to combine row letter with col number. 
    tidy_files[file_name]= remove_cols(df) # pass the df into the remove cols function to remove col for row letter and col number and add to dictionary with asoociated file name. 

tidy_files

{'cell_number':     cell_number well_id_96
 0       1000000         A1
 1       1000000         B1
 2       1000000         C1
 3       1000000         D1
 4       1000000         E1
 5       1000000         F1
 6       1000000         G1
 7       1000000         H1
 8       1000000         A2
 9       1000000         B2
 10      1000000         C2
 11      1000000         D2
 12      1000000         E2
 13      1000000         F2
 14      1000000         G2
 15      1000000         H2
 16      1000000         A3
 17      1000000         B3
 18      1000000         C3
 19      1000000         D3
 20      1000000         E3
 21      1000000         F3
 22      1000000         G3
 23      1000000         H3
 24      1000000         A4
 25      1000000         B4
 26      1000000         C4
 27      1000000         D4
 28      1000000         E4
 29      1000000         F4
 ..          ...        ...
 66      1000000         C9
 67      1000000         D9
 68      1000000         E9
 69  

In [9]:
dfs=list(tidy_files.values()) # create list of tidy dataframes
dfs_merged=reduce(lambda left,right: pd.merge(left,right, on="well_id_96"), dfs) # merge all dataframes together on well_id
dfs_merged.to_csv("{}/metadata.csv".format(path_to_plates))
dfs_merged

,cell_number,well_id_96,lonza_nucleofection_buffer
0,1000000,A1,Buffer_P1
1,1000000,B1,Buffer_P1
2,1000000,C1,Buffer_P1
3,1000000,D1,Buffer_P1
4,1000000,E1,Buffer_P1
5,1000000,F1,Buffer_P1
6,1000000,G1,Buffer_P1
7,1000000,H1,Buffer_P1
8,1000000,A2,Buffer_P1
9,1000000,B2,Buffer_P1


In [10]:
path_to_coords='/Users/giana.cirolia/Desktop/Nucleofection_Pipelne_Code/coord_maps'
path_to_coords

'/Users/giana.cirolia/Desktop/Nucleofection_Pipelne_Code/coord_maps'

In [11]:
all_coords=glob.glob("{}/*.csv".format(path_to_coords))
all_coords

['/Users/giana.cirolia/Desktop/Nucleofection_Pipelne_Code/coord_maps/coord1_384.csv',
 '/Users/giana.cirolia/Desktop/Nucleofection_Pipelne_Code/coord_maps/coord2_384.csv']

In [12]:
path_to_i7_384='/Users/giana.cirolia/Desktop/Nucleofection_Pipelne_Code/i7_index_384'
path_to_i5_384='/Users/giana.cirolia/Desktop/Nucleofection_Pipelne_Code/i5_index_384'

In [13]:
all_i7_index_384=glob.glob("{}/*.csv".format(path_to_i7_384))
all_i5_index_384=glob.glob("{}/*.csv".format(path_to_i5_384))
all_i5_index_384

['/Users/giana.cirolia/Desktop/Nucleofection_Pipelne_Code/i5_index_384/i5_index_384.csv']

In [14]:
dict_all_i7_index_384={}

for file in all_i7_index_384:
    file_name=os.path.splitext(os.path.basename(file))[0]
    df=pd.read_csv(file)
    dict_all_i7_index_384[file_name]=df

dict_all_i7_index_384

{'i7_index_384':     i7_index_384 I7_Index_ID i7_index_sequence
 0             A1    i7-00001          CTGATCGT
 1             A2    i7-00002          ACTCTCGA
 2             A3    i7-00003          TGAGCTAG
 3             A4    i7-00004          GAGACGAT
 4             A5    i7-00005          CTTGTCGA
 5             A6    i7-00006          TTCCAAGG
 6             A7    i7-00049          CGCATGAT
 7             A8    i7-00008          ACGGAACA
 8             A9    i7-00009          CGGCTAAT
 9            A10    i7-00010          ATCGATCG
 10           A11    i7-00011          GCAAGATC
 11           A12    i7-00012          GCTATCCT
 12           A13    i7-00013          TACGCTAC
 13           A14    i7-00014          TGGACTCT
 14           A15    i7-00015          AGAGTAGC
 15           A16    i7-00016          ATCCAGAG
 16           A17    i7-00017          GACGATCT
 17           A18    i7-00018          AACTGAGC
 18           A19    i7-00019          CTTAGGAC
 19           A20    i7-

In [17]:
dict_all_i5_index_384={}

for file in all_i5_index_384:
    file_name=os.path.splitext(os.path.basename(file))[0]
    df=pd.read_csv(file)
    dict_all_i5_index_384[file_name]=df
dict_all_i5_index_384['i5_index_384']

,i5_index_384,i5_index_ID,i5_index_sequence
0,A1,i5-00001,ACGATCAG
1,A2,i5-00002,TCGAGAGT
2,A3,i5-00003,CTAGCTCA
3,A4,i5-00004,ATCGTCTC
4,A5,i5-00005,TCGACAAG
5,A6,i5-00006,CCTTGGAA
6,A7,i5-00049,ATCATGCG
7,A8,i5-00008,TGTTCCGT
8,A9,i5-00009,ATTAGCCG
9,A10,i5-00010,CGATCGAT


In [19]:
dict_of_coords={}

for file in all_coords:
    file_name=os.path.splitext(os.path.basename(file))[0]
    df=pd.read_csv(file)
    dict_of_coords[file_name]=df

dict_of_coords

{'coord1_384':    well_id_96 coord1_384
 0          A1         A1
 1          A2         A3
 2          A3         A5
 3          A4         A7
 4          A5         A9
 5          A6        A11
 6          A7        A13
 7          A8        A15
 8          A9        A17
 9         A10        A19
 10        A11        A21
 11        A12        A23
 12         B1         C1
 13         B2         C3
 14         B3         C5
 15         B4         C7
 16         B5         C9
 17         B6        C11
 18         B7        C13
 19         B8        C15
 20         B9        C17
 21        B10        C19
 22        B11        C21
 23        B12        C23
 24         C1         E1
 25         C2         E3
 26         C3         E5
 27         C4         E7
 28         C5         E9
 29         C6        E11
 ..        ...        ...
 66         F7        K13
 67         F8        K15
 68         F9        K17
 69        F10        K19
 70        F11        K21
 71        F12        K2

In [23]:
i7=dict_all_i7_index_384["i7_index_384"]
coord1=dict_of_coords["coord1_384"]
coord1_i7_merged=coord1.merge(i7, left_on="coord1_384", right_on="i7_index_384")
coord1_i7_merged

,well_id_96,coord1_384,i7_index_384,I7_Index_ID,i7_index_sequence
0,A1,A1,A1,i7-00001,CTGATCGT
1,A2,A3,A3,i7-00003,TGAGCTAG
2,A3,A5,A5,i7-00005,CTTGTCGA
3,A4,A7,A7,i7-00049,CGCATGAT
4,A5,A9,A9,i7-00009,CGGCTAAT
5,A6,A11,A11,i7-00011,GCAAGATC
6,A7,A13,A13,i7-00013,TACGCTAC
7,A8,A15,A15,i7-00015,AGAGTAGC
8,A9,A17,A17,i7-00017,GACGATCT
9,A10,A19,A19,i7-00019,CTTAGGAC


In [28]:

new=coord1_i7_merged.merge(dfs_merged, left_on="well_id_96", right_on="well_id_96")
new


,well_id_96,coord1_384,i7_index_384,I7_Index_ID,i7_index_sequence,cell_number,lonza_nucleofection_buffer
0,A1,A1,A1,i7-00001,CTGATCGT,1000000,Buffer_P1
1,A2,A3,A3,i7-00003,TGAGCTAG,1000000,Buffer_P1
2,A3,A5,A5,i7-00005,CTTGTCGA,1000000,Buffer_P1
3,A4,A7,A7,i7-00049,CGCATGAT,1000000,Buffer_P1
4,A5,A9,A9,i7-00009,CGGCTAAT,1000000,Buffer_P2
5,A6,A11,A11,i7-00011,GCAAGATC,1000000,Buffer_P2
6,A7,A13,A13,i7-00013,TACGCTAC,1000000,Buffer_P2
7,A8,A15,A15,i7-00015,AGAGTAGC,1000000,Buffer_P2
8,A9,A17,A17,i7-00017,GACGATCT,1000000,Buffer_P3
9,A10,A19,A19,i7-00019,CTTAGGAC,1000000,Buffer_P3
